In [35]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
distrito_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Distritos\Hogares_RegimenDeTenencia_Distritos_Censo_2011.csv"
distrito_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Distritos\Hogares_RegimenDeTenencia_Distritos_Censo_2021.csv"
atributo = "Regimen de Tenencia"
valores = "Hogares"
valoresTotal = "Total_hogares_distrito"
valoresPorcentaje = "Hogares %"
cartografia_distrito_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaDistritoMunicipal-25830\da_cartografiaDistritoMunicipalPolygon.shp"
export_distrito_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Distritos\2011_Distritos_RegimenDeTenencia.csv"
export_distrito_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\08 - Regimen de Tenencia\Distritos\2021_Distritos_RegimenDeTenencia.csv"

## Lectura inicial de datos lugar de nacimiento
df_distrito_2011 =  pd.read_csv(distrito_2011_csv_path,sep=";")
df_distrito_2021 =  pd.read_csv(distrito_2021_csv_path,sep=";")
df_cartografia_distrito = gpd.read_file(cartografia_distrito_shp_path)

# Ver las columnas
#print(df_distrito_2011.columns)
#print(df_distrito_2021.columns)

# Inspeccionar los primeros registros
#print(df_distrito_2011.head())
#print(df_distrito_2021.head())

#Cambio el nombre de la columna 'Distrito ' a Distrito de residencia
df_distrito_2021.rename(columns={'Distrito ': 'Distrito de residencia'}, inplace=True)

#Se realiza una normalizacion de los csv  
df_distrito_2011_long = pd.melt(df_distrito_2011, id_vars=["Municipio de residencia","Distrito de residencia"], var_name=atributo, value_name=valores)
#print(df_distrito_2011_long.columns)
#print(df_distrito_2011_long.head())
df_distrito_2021_long = pd.melt(df_distrito_2021, id_vars=["Municipio de residencia","Distrito de residencia"], var_name=atributo, value_name=valores)
#print(df_distrito_2021_long.columns)
#print(df_distrito_2021_long.head())

## Eliminar filas ##
#Eliminar filas de municipio Total
df_distrito_2011_long = df_distrito_2011_long[df_distrito_2011_long['Municipio de residencia'] != 'Total']
df_distrito_2021_long = df_distrito_2021_long[df_distrito_2021_long['Municipio de residencia'] != 'Total']
#Eliminar filas de Distrito de residencia Total
df_distrito_2011_long = df_distrito_2011_long[df_distrito_2011_long['Distrito de residencia'] != 'Total']
df_distrito_2021_long = df_distrito_2021_long[df_distrito_2021_long['Distrito de residencia'] != 'Total']

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_distrito_2011_long[valores].isna()).sum()
num_filas_con_asterisco_2011 = (df_distrito_2011_long[valores] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_distrito_2011_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#0
#print(num_filas_contiene_asterisco_2011)#0
num_filas_en_blanco_2021 = (df_distrito_2021_long[valores].isna()).sum()
num_filas_con_asterisco_2021 = (df_distrito_2021_long[valores] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_distrito_2021_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

#print(df_distrito_2011_long[atributo].unique())#4
#print(df_distrito_2021_long[atributo].unique())#4
df_distrito_2011_long = df_distrito_2011_long[df_distrito_2011_long[atributo] != 'Total']
df_distrito_2021_long = df_distrito_2021_long[df_distrito_2021_long[atributo] != 'Total']

## Añadir columnas ##
#Añadir columna NUMERO con el numero de distrito
df_distrito_2011_long["NUMERO"] = df_distrito_2011_long["Distrito de residencia"].str[-2:].astype(float)
df_distrito_2021_long["NUMERO"] = df_distrito_2021_long["Distrito de residencia"].str[-2:].astype(float)

#print(df_distrito_2011_long.head())
#print(df_distrito_2021_long.head())

## Gestion de Nulos y valores extraños ##
# Limpiar los datos: eliminar el asterisco al final, pero dejar intacto el '*'
df_distrito_2011_long[valores] = df_distrito_2011_long[valores].replace(r'(\d+)\*$', r'\1', regex=True)
df_distrito_2021_long[valores] = df_distrito_2021_long[valores].replace(r'(\d+)\*$', r'\1', regex=True)
#Sustituimos los valores * por NA
df_distrito_2011_long[valores] = df_distrito_2011_long[valores].replace('*', np.nan)
df_distrito_2021_long[valores] = df_distrito_2021_long[valores].replace('*', np.nan)
#Columna Personas como float
df_distrito_2011_long[valores] = df_distrito_2011_long[valores].astype(float)
df_distrito_2021_long[valores] = df_distrito_2021_long[valores].astype(float)

# Diccionario de mapeo 2011 a 2021
mapping_dict_2011_2021 = {
    'Total': 'Total',
    'Al menos un miembro es arrendatario de toda o parte de la vivienda': 'En alquiler',
    'Al menos un miembro es propietario de la vivienda': 'En propiedad',
    'Otro régimen de tenencia': 'Otro régimen de tenencia'
}

df_distrito_2011_long[atributo] = df_distrito_2011_long[atributo].map(mapping_dict_2011_2021)

##Calcular el total de personas por distrito y el porcentaje 
df_distrito_2011_long[valoresTotal] = df_distrito_2011_long.groupby('Distrito de residencia')[valores].transform('sum')
df_distrito_2021_long[valoresTotal] = df_distrito_2021_long.groupby('Distrito de residencia')[valores].transform('sum')
#Calcular porcentaje sobre el total
df_distrito_2011_long[valoresPorcentaje] = (df_distrito_2011_long[valores] / df_distrito_2011_long[valoresTotal])
df_distrito_2021_long[valoresPorcentaje] = (df_distrito_2021_long[valores] / df_distrito_2021_long[valoresTotal])
#print(df_distrito_2011_long[df_distrito_2011_long['NUMERO'] == 1])
#print(df_distrito_2011_long[df_distrito_2011_long['NUMERO'] == 3])
#print(df_distrito_2011_long[df_distrito_2011_long['NUMERO'] == 5])
#print(df_distrito_2021_long[df_distrito_2021_long['NUMERO'] == 1])

#Pivotar la tabla de nuevo para crear variables, una por categoria con el % ; una fila por distrito
df_distrito_2011_long  = df_distrito_2011_long.pivot(index='NUMERO', columns=atributo, values=valoresPorcentaje)
df_distrito_2021_long  = df_distrito_2021_long.pivot(index='NUMERO', columns=atributo, values=valoresPorcentaje)

print(df_distrito_2011_long)
print(df_distrito_2021_long)

##Export 3 variables
df_distrito_2011_long.to_csv(export_distrito_2011_csv_path,index=True,sep=";")
df_distrito_2021_long.to_csv(export_distrito_2021_csv_path,index=True,sep=";")

Regimen de Tenencia  En alquiler  En propiedad  Otro régimen de tenencia
NUMERO                                                                  
1.0                     0.340956      0.584200                  0.074844
2.0                     0.131172      0.778767                  0.090061
3.0                     0.422961      0.435045                  0.141994
4.0                     0.225352      0.683881                  0.090767
5.0                     0.512535      0.426184                  0.061281
6.0                     0.143015      0.779620                  0.077365
7.0                     0.131561      0.790646                  0.077793
8.0                     0.093466      0.846624                  0.059910
9.0                     0.168439      0.753493                  0.078068
10.0                    0.107254      0.844586                  0.048161
11.0                    0.057320      0.883036                  0.059644
Regimen de Tenencia  En alquiler  En propiedad  Otr